<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/serialisation_wi_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets evaluate rouge_score
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import pandas as pd
import numpy as np

In [8]:
import torch
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [9]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'

# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

In [10]:
# initialize the model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
)

model.eval();

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


### Few Shot inference on serialised data

In [152]:
serialised_data = pd.read_csv("df_combined_table_ser_sample3000.csv")
serialised_data.head()

,pre_text,post_text,filename,table_ori,table,id,dataset,prompt,serialized_text_gpt35
0,"['in addition , the company has reclassified t...",['.'],RE/2013/page_109.pdf,"[['', 'Years Ended December 31,'], ['(Dollars ...","[['( dollars in thousands )', 'years ended dec...",RE/2013/page_109.pdf-3,train,"\n ### Table:[['', 'Years Ended December 31...",The table provides information on the reinsura...
1,"['related employer payroll tax costs ) .', 'th...",['there was an insignificant amount of potenti...,CDW/2014/page_93.pdf,"[['', 'Years Ended December 31,'], ['(in milli...","[['( in millions )', 'years ended december 31 ...",CDW/2014/page_93.pdf-1,train,"\n ### Table:[['', 'Years Ended December 31...",The table provided shows the number of weighte...
2,"['other long term debt in december 2012 , the ...","['interest expense , net was $ 34.5 million , ...",UAA/2017/page_86.pdf,"[['2018', '$27,000'], ['2019', '63,000'], ['20...","[['2018', '$ 27000'], ['2019', '63000'], ['202...",UAA/2017/page_86.pdf-2,train,"\n ### Table:[['2018', '$27,000'], ['2019',...",The table above lists the scheduled maturities...
3,['stock total return performance the following...,['the stock price performance included in this...,HUM/2017/page_45.pdf,"[['', '12/31/2012', '12/31/2013', '12/31/2014'...","[['', '12/31/2012', '12/31/2013', '12/31/2014'...",HUM/2017/page_45.pdf-3,train,"\n ### Table:[['', '12/31/2012', '12/31/201...",The table provided compares the performance of...
4,['contingencies we are exposed to certain know...,['( 1 ) interest payments on our debt are base...,IQV/2018/page_59.pdf,"[['(in millions)', '2019', '2020 - 2021', '202...","[['( in millions )', '2019', '2020 - 2021', '2...",IQV/2018/page_59.pdf-3,train,"\n ### Table:[['(in millions)', '2019', '20...",The table provides information on the total am...


In [153]:
serialised_data.shape

(3000, 9)

In [154]:
def fewshot_instruction(example,i):
    context = example['table_ori']
    question = "Serialise the table above into regular text."
    answer = example['serialized_text_gpt35']
    return f"""### Instruction:
Answer the question using the information from the context below. There are three examples given below, followed by the question you need to answer.

### Example {i}

### Context:
{context.strip()}

### Question:
{question.strip()}

### Answer:
{answer}
""".strip()

In [155]:
fewshot_prompt=""
c=1
for i in np.random.randint(0, len(serialised_data),3):
  print (i,c)
  fewshot_prompt +=fewshot_instruction(serialised_data.iloc[i],c)
  c+=1
print (fewshot_prompt)

2977 1
2466 2
123 3
### Instruction:
Answer the question using the information from the context below. There are three examples given below, followed by the question you need to answer.

### Example 1

### Context:
[['', 'Year Ended December'], ['<i>in millions</i>', '2013', '2012', '2011'], ['Residential mortgages', '$29,772', '$33,755', '$40,131'], ['Commercial mortgages', '6,086', '300', '—'], ['Other financial assets', '—', '—', '269'], ['Total', '$35,858', '$34,055', '$40,400'], ['Cash flows on retained interests', '$ 249', '$ 389', '$ 569']]

### Question:
Serialise the table above into regular text.

### Answer:
The table provides information on the amount of residential and commercial mortgages, as well as other financial assets for the years 2011 to 2013. In 2013, residential mortgages amounted to $29,772 million, which is lower than the previous year's total of $33,755 million. Commercial mortgages, on the other hand, increased significantly from $300 million in 2012 to $6,08

In [156]:
serialised_data.columns

Index(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'id',
       'dataset', 'prompt', 'serialized_text_gpt35'],
      dtype='object')

In [157]:
def prompt_instruction(example, cat):
    context = example['table_ori']
    question = "Serialise the table above into regular text."
    answer = example['serialized_text_gpt35']
    if cat=="fewshot":
      cat_inst = "Now given the above three examples, answer the question using the information from the context below."
    else:
      cat_inst = "Answer the question using the information from the context below."
    return f"""### Instruction:
{cat_inst}.

### Context:
{context.strip()}

### Question:
{question.strip()}

### Answer:
""".strip()

In [158]:
from tqdm.notebook import tqdm

In [159]:
results_list=[]
# fewshot_prompts = []
# zeroshow_prompts = []
for row in tqdm(serialised_data.sample(frac=0.2).iterrows()):
  fewshot_prompt_i = fewshot_prompt + prompt_instruction(row[1],cat="fewshot")
  zeroshot_prompt_i = prompt_instruction(row[1],cat="zero")
  result_fewshot_i = generate_text(fewshot_prompt_i)
  result_zeroshot_i = generate_text(zeroshot_prompt_i)
  format_fewshot_i = result_fewshot_i[0]["generated_text"].split("### Answer:")[-1].strip()
  format_zeroshot_i = result_zeroshot_i[0]["generated_text"].split("### Answer:")[-1].strip()
  result_dict_i={"id":row[1]["id"],\
                 "benchmark":row[1]["serialized_text_gpt35"],\
                 "fewshot_serialise_llama":format_fewshot_i,\
                 "zeroshot_serialise_llama":format_zeroshot_i}
  results_list.append(result_dict_i)

0it [00:00, ?it/s]

In [ ]:
# generate_text(fewshot_prompts)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [77]:
results_df = pd.DataFrame(results_list)
results_df.head()

,id,benchmark,serialise_llama
0,RE/2013/page_109.pdf-3,The table provides information on the reinsura...,"According to the table, the reinsurance receiv..."
1,CDW/2014/page_93.pdf-1,The table provided shows the number of weighte...,The table shows the weighted-average number of...
2,UAA/2017/page_86.pdf-2,The table above lists the scheduled maturities...,"Based on the information provided, the total s..."
3,HUM/2017/page_45.pdf-3,The table provided compares the performance of...,"The table shows the value of HUM, S&P 500, and..."
4,IQV/2018/page_59.pdf-3,The table provides information on the total am...,"The table shows the long-term debt, operating ..."


In [ ]:
results_df.to_csv("serialisation_fewshot_zeroshot.csv")

## Bleu METRIC

In [106]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [112]:
import evaluate

In [113]:
# rouge = evaluate.load('rouge')
# predictions = ["hello there", "general kenobi"]
# references = ["hello there", "general kenobi"]
# results = rouge.compute(predictions=predictions,references=references)
# results

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

## Zero zhot metric

In [114]:
predictions = results_df.zeroshot_serialise_llama.tolist()
references = results_df.benchmark.tolist()
results = rouge.compute(predictions=predictions,references=references)
results

{'rouge1': 0.48905850233861103,
 'rouge2': 0.27410018839883094,
 'rougeL': 0.36691778465772273,
 'rougeLsum': 0.3983119998023441}

## Few Shot Metric

In [ ]:
predictions = results_df.fewshot_serialise_llama.tolist()
references = results_df.benchmark.tolist()
results = rouge.compute(predictions=predictions,references=references)
results